In [0]:
from pyspark.sql.functions import col, lit, expr, when

from pyspark.sql.types import *

from datetime import datetime

import time

 

# Define schema

nyc_schema = StructType([

  StructField('Vendor', StringType(), True),

  StructField('Pickup_DateTime', TimestampType(), True),

  StructField('Dropoff_DateTime', TimestampType(), True),

  StructField('Passenger_Count', IntegerType(), True),

  StructField('Trip_Distance', DoubleType(), True),

  StructField('Pickup_Longitude', DoubleType(), True),

  StructField('Pickup_Latitude', DoubleType(), True),

  StructField('Rate_Code', StringType(), True),

  StructField('Store_And_Forward', StringType(), True),

  StructField('Dropoff_Longitude', DoubleType(), True),

  StructField('Dropoff_Latitude', DoubleType(), True),

  StructField('Payment_Type', StringType(), True),

  StructField('Fare_Amount', DoubleType(), True),

  StructField('Surcharge', DoubleType(), True),

  StructField('MTA_Tax', DoubleType(), True),

  StructField('Tip_Amount', DoubleType(), True),

  StructField('Tolls_Amount', DoubleType(), True),

  StructField('Total_Amount', DoubleType(), True)

])

 

rawDF = spark.read.format('csv').options(header=True).schema(nyc_schema).load("dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-12.csv.gz")\

rawDF.show(15)







+------+-------------------+-------------------+---------------+-------------+----------------+---------------+---------+-----------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|Vendor|    Pickup_DateTime|   Dropoff_DateTime|Passenger_Count|Trip_Distance|Pickup_Longitude|Pickup_Latitude|Rate_Code|Store_And_Forward|Dropoff_Longitude|Dropoff_Latitude|Payment_Type|Fare_Amount|Surcharge|MTA_Tax|Tip_Amount|Tolls_Amount|Total_Amount|
+------+-------------------+-------------------+---------------+-------------+----------------+---------------+---------+-----------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|     1|2019-12-01 00:26:58|2019-12-01 00:41:45|              1|          4.2|             1.0|           null|      142|              116|              2.0|            14.5|           3|        0.5|      0.0|    0.0|       0.3|       

In [0]:
from pyspark.sql.functions import col, lit, expr, when

from pyspark.sql.types import *

from datetime import datetime

import time

# 3 
corruptedDF = rawDF.withColumn("Tip_Amount", when(col("Tip_Amount") == 0.3, lit("trzydziesiate")).otherwise(col("Tip_Amount")))


corrupted_file_path = "dbfs:/tmp/corrupted_tips.csv"
corruptedDF.write.mode("overwrite").option("header", "true").csv(corrupted_file_path)

read_modes = ["PERMISSIVE", "DROPMALFORMED", "FAILFAST"]

for mode in read_modes:
    print(f"\n Odczyt w trybie: {mode}")
    try:
        df = spark.read.format("csv").option("header", "true").option("mode", mode).schema(nyc_schema).load(corrupted_file_path)
        df.show(5)
    except Exception as e:
        print(f" Błąd w trybie {mode}: {e}")


In [0]:
from pyspark.sql.functions import col, lit, expr, when

from pyspark.sql.types import *

from datetime import datetime

import time

# 4 

parquet_path = "dbfs:/tmp/nyc_taxi.parquet"
json_path = "dbfs:/tmp/nyc_taxi.json"

rawDF.write.mode("overwrite").parquet(parquet_path)
rawDF.write.mode("overwrite").json(json_path)

#sprawdzenie


print("\n Dane z Parquet:")
parquetDF = spark.read.parquet(parquet_path)
parquetDF.show(5)

# Odczyt JSON
print("\n Dane z JSON:")
jsonDF = spark.read.json(json_path)
jsonDF.show(5)



 Dane z Parquet:
+------+-------------------+-------------------+---------------+-------------+----------------+---------------+---------+-----------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|Vendor|    Pickup_DateTime|   Dropoff_DateTime|Passenger_Count|Trip_Distance|Pickup_Longitude|Pickup_Latitude|Rate_Code|Store_And_Forward|Dropoff_Longitude|Dropoff_Latitude|Payment_Type|Fare_Amount|Surcharge|MTA_Tax|Tip_Amount|Tolls_Amount|Total_Amount|
+------+-------------------+-------------------+---------------+-------------+----------------+---------------+---------+-----------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|     1|2019-12-01 00:26:58|2019-12-01 00:41:45|              1|          4.2|             1.0|           null|      142|              116|              2.0|            14.5|           3|        0.5|      0.0|    0.0|